# Import necessary libraries

In [5]:
import pandas as pd
import glob
import os 
import json

# Combine all the data collected into one csv file

In [6]:
current_directory = os.path.dirname(os.path.realpath('preprocessing.ipynb'))
database_directory=os.path.join(current_directory,'..',"Database")
os.chdir(database_directory)

extension='txt'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

data_list=[]

for file in all_filenames:
    with open(file,'r') as f:
        file_content=f.read()
        for line in file_content.split('\n'):
            if line.strip(): 
                line=line.replace("'","\"")
                data_dict=json.loads(line)
                data_list.append(data_dict)

combined_data=pd.DataFrame(data_list)
combined_data.to_csv("combined_data.csv", index=False, encoding='utf-8-sig')


# Data cleaning

### Data overview

In [7]:
dataframe = pd.read_csv('..\Database\combined_data.csv')
dataframe


,brand,dtime,flytime,atime,price,crawl_date,dcity,acity,flight_type,class_type,ddate,fcode
0,Vietravel Airlines,23:59,2g 16p,02:15,1377000,2023-12-06,ho-chi-minh-city,hanoi,oneway,economy,2023-12-06,NaN
1,VietJet Air,22:25,2g 10p,00:35,1447000,2023-12-06,ho-chi-minh-city,hanoi,oneway,economy,2023-12-06,NaN
2,VietJet Air,23:30,2g 5p,01:35,1447000,2023-12-06,ho-chi-minh-city,hanoi,oneway,economy,2023-12-06,NaN
3,VietJet Air,23:40,2g 10p,01:50,1447000,2023-12-06,ho-chi-minh-city,hanoi,oneway,economy,2023-12-06,NaN
4,VietJet Air,21:40,2g 10p,23:50,1563000,2023-12-06,ho-chi-minh-city,hanoi,oneway,economy,2023-12-06,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
78492,VietJet Air,11:25,8g 20p,19:45,6217000,2023-12-26,dien-bien-phu,haiphong,oneway,economy,2023-12-25,"['VJ299', 'VJ270']"
78493,VietJet Air,11:25,8g 20p,19:45,10091000,2023-12-26,dien-bien-phu,haiphong,oneway,economy,2023-12-25,"['VN1803', 'VN7205', 'VJ286']"
78494,VietJet Air,11:25,8g 20p,19:45,10387000,2023-12-26,dien-bien-phu,haiphong,oneway,economy,2023-12-25,"['VN1803', 'VN7205', 'VJ284']"
78495,VietJet Air,11:25,8g 20p,19:45,10402000,2023-12-26,dien-bien-phu,haiphong,oneway,economy,2023-12-25,"['VN1803', 'VN7205', 'VN1182']"


**The description of each column:**

> brand: The airline company.

> dtime: The departure time of the flight.

> flytime: The total duration of the flight.

> atime: The arrival time of the flight.

> price: The price of the flight ticket.

> dcity: The city from which the flight departs.

> acity: The city where the flight lands.

> flight type: The type of flight (e.g., one-way, round-trip).

> class_type: The class of the flight ticket (e.g., economy, business).

> ddate: The date of the flight.

> crawl date: The date when the flight information was collected or "crawled" from the source.

> fcode: The flight number or code.


## Checking missing values

In [8]:
null_values = dataframe.isnull().sum()
print("Null Values:\n", null_values)


Null Values:
 brand           0
dtime           0
flytime         0
atime           0
price           0
crawl_date      0
dcity           0
acity           0
flight_type     0
class_type      0
ddate           0
fcode          52
dtype: int64


After reviewing the missing values in the dataset, it is observed that the only column with missing data is fcode, which has 52 missing entries. However, considering the real-world context of how customers choose flights, we believe that the flight code is not a significant factor influencing their purchasing decisions. Therefore, I have decided not to perform any imputation for the missing values in the fcode column and instead plan to drop this column from the dataset as part of the data cleaning process.

## Dropping unnecessary columns

In [13]:
for column in dataframe.columns:
    unique_values = dataframe[column].unique()
    print(f"Column Name: {column}")
    print(f"Number of Unique Values: {len(unique_values)}")

Column Name: brand
Number of Unique Values: 18
Column Name: dtime
Number of Unique Values: 215
Column Name: flytime
Number of Unique Values: 432
Column Name: atime
Number of Unique Values: 218
Column Name: price
Number of Unique Values: 14335
Column Name: crawl_date
Number of Unique Values: 13
Column Name: dcity
Number of Unique Values: 18
Column Name: acity
Number of Unique Values: 18
Column Name: flight_type
Number of Unique Values: 1
Column Name: class_type
Number of Unique Values: 2
Column Name: ddate
Number of Unique Values: 13
Column Name: fcode
Number of Unique Values: 26049


The 'crawl_date' column, which indicates the date the data was collected, is not essential for our analysis and can be dropped. Similarly, the 'f_code' column, representing the flight code, is not typically a significant factor in customer decision-making and can also be subject to change post-booking, so we can remove it as well. Lastly, the 'flight_type' column only contains one unique value, meaning all data entries share the same value for this attribute. This lack of variation makes it uninformative for our analysis, so we can drop this column too.

In [14]:
df=dataframe.drop(['crawl_date','fcode','flight_type'], axis=1)
df

,brand,dtime,flytime,atime,price,dcity,acity,class_type,ddate
0,Vietravel Airlines,23:59,2g 16p,02:15,1377000,ho-chi-minh-city,hanoi,economy,2023-12-06
1,VietJet Air,22:25,2g 10p,00:35,1447000,ho-chi-minh-city,hanoi,economy,2023-12-06
2,VietJet Air,23:30,2g 5p,01:35,1447000,ho-chi-minh-city,hanoi,economy,2023-12-06
3,VietJet Air,23:40,2g 10p,01:50,1447000,ho-chi-minh-city,hanoi,economy,2023-12-06
4,VietJet Air,21:40,2g 10p,23:50,1563000,ho-chi-minh-city,hanoi,economy,2023-12-06
...,...,...,...,...,...,...,...,...,...
78492,VietJet Air,11:25,8g 20p,19:45,6217000,dien-bien-phu,haiphong,economy,2023-12-25
78493,VietJet Air,11:25,8g 20p,19:45,10091000,dien-bien-phu,haiphong,economy,2023-12-25
78494,VietJet Air,11:25,8g 20p,19:45,10387000,dien-bien-phu,haiphong,economy,2023-12-25
78495,VietJet Air,11:25,8g 20p,19:45,10402000,dien-bien-phu,haiphong,economy,2023-12-25


## Remove Duplicated Data


Based on the data collected over a period of time, we have observed that there are numerous instances where all flight details, including departure city, arrival city, departure time, arrival time, and price, remain the same except for the departure date. To streamline our dataset and remove redundancy, we have decided to identify and eliminate these duplicate entries. This will ensure that our data analysis is more accurate and efficient, as we will not be considering the same flight information multiple times.

In [17]:
important_columns = ['brand','dtime', 'flytime','atime','price', 'dcity', 'acity', 'class_type']
df2 = df.drop_duplicates(subset=important_columns, keep='first')
num_removed_rows = len(df) - len(df2)
print(f"Removed {num_removed_rows} duplicate rows.")
df2.reset_index(drop=True, inplace=True)
print(df2)

Removed 24605 duplicate rows.
                    brand  dtime flytime  atime     price             dcity  \
0      Vietravel Airlines  23:59  2g 16p  02:15   1377000  ho-chi-minh-city   
1             VietJet Air  22:25  2g 10p  00:35   1447000  ho-chi-minh-city   
2             VietJet Air  23:30   2g 5p  01:35   1447000  ho-chi-minh-city   
3             VietJet Air  23:40  2g 10p  01:50   1447000  ho-chi-minh-city   
4             VietJet Air  21:40  2g 10p  23:50   1563000  ho-chi-minh-city   
...                   ...    ...     ...    ...       ...               ...   
53887         VietJet Air  11:25  8g 20p  19:45   6217000     dien-bien-phu   
53888         VietJet Air  11:25  8g 20p  19:45  10091000     dien-bien-phu   
53889         VietJet Air  11:25  8g 20p  19:45  10387000     dien-bien-phu   
53890         VietJet Air  11:25  8g 20p  19:45  10402000     dien-bien-phu   
53891         VietJet Air  11:25  8g 20p  19:45  11256000     dien-bien-phu   

          acity class

After removing duplicated data, the size of the datasets went from 78497 to 53892. There are a total of 24605 data removed.

## Converting data columns

### 1. Convert data

In [19]:
df2.drop(['ddate'],axis=1).describe(include="object").T

,count,unique,top,freq
brand,53892,18,Vietnam Airlines,27773
dtime,53892,215,14:40,1616
flytime,53892,432,1g 20p,1366
atime,53892,218,17:55,1928
dcity,53892,18,phu-quoc-island,4499
acity,53892,18,da-nang,3772
class_type,53892,2,economy,41028


The values in the flytime column now is in form 'Xg Yp'. We need to convert it into a numberical form. 

In [20]:
def convert_flytime_to_hours(flytime):
    try:
        hours, minutes = map(int, flytime.replace('g', '').replace('p', '').split())
        total_hours = hours + minutes/60
        return total_hours
    except ValueError:
        return None

df2.loc[:, 'flytime_hours'] = df2.loc[:,'flytime'].apply(convert_flytime_to_hours)
print(df2[['flytime', 'flytime_hours']])

      flytime  flytime_hours
0      2g 16p       2.266667
1      2g 10p       2.166667
2       2g 5p       2.083333
3      2g 10p       2.166667
4      2g 10p       2.166667
...       ...            ...
53887  8g 20p       8.333333
53888  8g 20p       8.333333
53889  8g 20p       8.333333
53890  8g 20p       8.333333
53891  8g 20p       8.333333

[53892 rows x 2 columns]


C:\Users\2053s\AppData\Local\Temp\ipykernel_24136\2599892295.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:, 'flytime_hours'] = df2.loc[:,'flytime'].apply(convert_flytime_to_hours)


In [24]:
df2=df2.drop(['flytime'],axis=1)
df2

,brand,dtime,atime,price,dcity,acity,class_type,ddate,flytime_hours
0,Vietravel Airlines,23:59,02:15,1377000,ho-chi-minh-city,hanoi,economy,2023-12-06,2.266667
1,VietJet Air,22:25,00:35,1447000,ho-chi-minh-city,hanoi,economy,2023-12-06,2.166667
2,VietJet Air,23:30,01:35,1447000,ho-chi-minh-city,hanoi,economy,2023-12-06,2.083333
3,VietJet Air,23:40,01:50,1447000,ho-chi-minh-city,hanoi,economy,2023-12-06,2.166667
4,VietJet Air,21:40,23:50,1563000,ho-chi-minh-city,hanoi,economy,2023-12-06,2.166667
...,...,...,...,...,...,...,...,...,...
53887,VietJet Air,11:25,19:45,6217000,dien-bien-phu,haiphong,economy,2023-12-25,8.333333
53888,VietJet Air,11:25,19:45,10091000,dien-bien-phu,haiphong,economy,2023-12-25,8.333333
53889,VietJet Air,11:25,19:45,10387000,dien-bien-phu,haiphong,economy,2023-12-25,8.333333
53890,VietJet Air,11:25,19:45,10402000,dien-bien-phu,haiphong,economy,2023-12-25,8.333333


### 2. Convert datatype

In [28]:
df2.dtypes

brand             object
dtime             object
atime             object
price              int64
dcity             object
acity             object
class_type        object
ddate             object
flytime_hours    float64
dtype: object

The ddate column is now in object type. We need to convert it into datetime type.

In [29]:
df2['ddate'] = pd.to_datetime(df2['ddate'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53892 entries, 0 to 53891
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   brand          53892 non-null  object        
 1   dtime          53892 non-null  object        
 2   atime          53892 non-null  object        
 3   price          53892 non-null  int64         
 4   dcity          53892 non-null  object        
 5   acity          53892 non-null  object        
 6   class_type     53892 non-null  object        
 7   ddate          53892 non-null  datetime64[ns]
 8   flytime_hours  48773 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 3.7+ MB


# Save the cleaned data

In [30]:
df2.to_csv('cleaned_data.csv', index=False)